In [1]:
import pandas
import MySQLdb
from PIL import Image
from matplotlib import pylab as plt
import numpy as np
from scipy import misc
from skimage import exposure
import os


In [2]:
#Define Database
db = MySQLdb.connect("menchelabdb.int.cemm.at","root","cqsr4h","ImageAnalysisDDI" )

In [3]:
def ensure_dir(file_path):
    '''
    Function to ensure a file path exists, else creates the path

    :param file_path:
    :return:
    '''
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)


In [4]:
def create_combined_Image(images,foci):
    '''
    Gets the 4 foci and combines them to one images. Perform an intensity rescaling

    :param images:
    :param foci:
    :return:
    '''


    for i in range(0,len(images)):
        v_min, v_max = np.percentile(images[i].flatten(), (0.1, 99.9))
        better_contrast = exposure.rescale_intensity(images[i],in_range=(v_min, v_max))
        images[i] = Image.fromarray(better_contrast,mode='RGB')


    new_im = Image.new('RGB', (2000, 1500))
    coordinates = {2: (0, 0), 3: (1000, 0), 1: (0, 750), 4: (1000, 750)}
    for image, c in zip(images, foci):
        image.thumbnail((1000, 1000))
        new_im.paste(image, coordinates[c])

    #plt.imshow(new_im)
    #plt.show()


    return new_im

In [5]:
def find_Well(plate,well):
    '''
    same as find single but to find a specific well on a plate e.g. A01 plate 1315001


    :param plate:
    :param well:
    :return:
    '''

    data = pandas.read_sql("select Image_FileName_DAPI,Image_FileName_Mitotracker,Image_FileName_BetaTubulin,Image_PathName_DAPI  from CLOMP_Per_Image where Image_Metadata_Plate = "+plate+" and Image_Metadata_Well like '"+well+"';", con=db)

    print data

    images = []
    foci = []
    for image, image2, image3, path in zip(data['Image_FileName_DAPI'], data['Image_FileName_BetaTubulin'],
                                           data['Image_FileName_Mitotracker'], data['Image_PathName_DAPI']):
        path = path.split('lab_menche')[1]

        foci.append(int(image.split('f0')[1].split('p')[0]))
        im = Image.open(
            "/Volumes/scratch/lab_menche" + path + "/" + image)
        im2 = Image.open(
            "/Volumes/scratch/lab_menche" + path + "/" + image2)
        im3 = Image.open(
            "/Volumes/scratch/lab_menche" + path + "/" + image3)

        
        k = np.array(im).astype(np.int16)
        k2 = np.array(im2).astype(np.int16)
        k3 = np.array(im3).astype(np.int16)
        
        rgb = np.dstack((k3, k2, k))
        r_rgb = misc.imresize(rgb, 100)

        test = misc.bytescale(r_rgb)

        images.append(test)
        # images.append(Image.fromarray(test, mode='RGB'))

    return create_combined_Image(images,foci)

In [6]:
def find_Single(drug1,conc):
    '''
    check the database for image URL and creates the image out of the 4 individual foci

    Limit DMSO to only 8 foci (=2 full images) as there are 32 * number of plates replicates

    :param drug1:
    :param conc:
    :return:
    '''


    #print drug1
    if drug1 != "DMSO":
        data = pandas.read_sql(
            "select Image_FileName_DAPI,Image_FileName_Mitotracker,Image_FileName_BetaTubulin,Image_PathName_DAPI  from CLOMP_Per_Image where Image_Metadata_ID_A  = '" + drug1 + "' and (Image_Metadata_Conc_A = "+str(conc)+" or  Image_Metadata_Conc_A like "+str(conc)+");",
            con=db)
    else:
        data = pandas.read_sql(
            "select Image_FileName_DAPI,Image_FileName_Mitotracker,Image_FileName_BetaTubulin,Image_PathName_DAPI  from CLOMP_Per_Image where Image_Metadata_ID_A  = '" + drug1 + "' and (Image_Metadata_Conc_A = "+str(conc)+" or  Image_Metadata_Conc_A like "+str(conc)+") limit 8;",
            con=db)

    #print  "select Image_FileName_DAPI,Image_FileName_Mitotracker,Image_FileName_BetaTubulin,Image_PathName_DAPI  from CLOMP_Per_Image where Image_Metadata_ID_A  = '" + drug1 + "' and (Image_Metadata_Conc_A <= "+str(conc)+" or  Image_Metadata_Conc_A like "+str(conc)+") limit 8;"
    if len(data) == 0:
        return None
        
    images = []
    foci = []
    for image, image2,image3,path in zip(data['Image_FileName_DAPI'], data['Image_FileName_BetaTubulin'], data['Image_FileName_Mitotracker'],data['Image_PathName_DAPI'] ):

        path = path.split('lab_menche')[1]

        print   "/Volumes/scratch/lab_menche"+path+"/" + image
        foci.append(int(image.split('f0')[1].split('p')[0]))
        im = Image.open(
            "/Volumes/scratch/lab_menche"+path+"/" + image)
        im2 = Image.open(
            "/Volumes/scratch/lab_menche"+path+"/" + image2)
        im3 = Image.open(
            "/Volumes/scratch/lab_menche" + path + "/" + image3)

        k = np.array(im).astype(np.int16)
        k2 = np.array(im2).astype(np.int16)
        k3 = np.array(im3).astype(np.int16)
        
        rgb = np.dstack((k3, k2, k))
        r_rgb = misc.imresize(rgb, 100)

        test = misc.bytescale(r_rgb)

        images.append(test)
        #images.append(Image.fromarray(test, mode='RGB'))


    combined_images = []
    for i in range(0,len(images),4):
        c = create_combined_Image(images[i:i+4], foci[i:i+4])
        combined_images.append(c)

    return combined_images

In [7]:
def get_AllConcentrations(drug):
    
    conentrations = pandas.read_sql("select distinct(Image_Metadata_Conc_A) from CLOMP_Per_Image where Image_Metadata_ID_A  = '" + drug + "';",con=db)['Image_Metadata_Conc_A'].values
    return conentrations
    


In [8]:
def getOneSingle(drug,conc):
    '''
    Function to return ALL replicates for a given drug and it's concentration (2 replicates in CLOMP)


    :param drug: Drug ID
    :param conc: Drug Concentration (20,2,0.2,0.02)
    :return:
    '''
    s1_images = find_Single(drug, conc)

    if s1_images == None:
        return None
    
    drug1_images = Image.new('RGB', (2020 * 2 - 10, 1500))


    for i, image in enumerate(s1_images):
        drug1_images.paste(image, (i * 2020, 0))


    #plt.imshow(drug1_images)
    #plt.tight_layout()
    #plt.show()

    #print 'Im here?
    ensure_dir('../Images/New_Singles/'+drug+'_'+str(conc)+'.png')
    plt.figure(figsize=(10,10))
    plt.imshow(drug1_images)
    plt.tight_layout()
    #plt.show()
    plt.savefig('../Images/New_Singles/'+drug+'_'+str(conc)+'.png',format='png',dpi=600)
    plt.close()

    return 1


In [10]:
significant_Drugs = ['CLOUD179', 'CLOUD177', 'CLOUD077', 'CLOUD063', 'CLOUD101', 'CLOUD102', 'CLOUD103', 'CLOUD104', 'CLOUD178', 'CLOUD194', 'CLOUD053', 
                     'CLOUD057', 'CLOUD112', 'CLOUD117', 'CLOUD115', 'CLOUD047', 'CLOUD046',
                     'CLOUD129', 'CLOUD209', 'CLOUD031', 'CLOUD141', 'CLOUD143', 'CLOUD027', 'CLOUD196', 'CLOUD199', 'CLOUD156', 'CLOUD152', 'CLOUD089']

In [14]:
significant_Drugs = ['CLOUD015','CLOUD003','CLOUD047']



if __name__ == "__main__":

    
    
    #getOneSingle('DMSO','0.0')

    #'''
    #fp = open('../Images/Possible_Images','r')
    #to_analyse = []
    #for line in fp:
    #    to_analyse.append(line.strip().split('\t'))

    to_analyse = significant_Drugs

    #fp = open('../Images/CLoMP_drugs_Isabel.csv', 'r')
    #possible_concentrations = ['0.15','1.5','15','105']

    for drug in to_analyse:
        print drug
        possible_concentrations = get_AllConcentrations(drug)
        
        for conc in possible_concentrations:
            print conc
            if os.path.isfile('../Images/New_Singles/'+drug+'_'+str(conc)+'.png'):
                continue
            
            
            result = getOneSingle(drug,conc)
            if result == None:
                print 'Not found in DB - Why?'
                
            #except:
            #print 'Problem with:'
            #print drug
            
    
    #to_analyse = []
    #for line in fp:
    #    for pc in possible_concentrations:
    #        to_analyse.append([line.strip(),pc])

    #print to_analyse
    #for image in to_analyse:
    #    try:
    #        getOneSingle(image[0],image[1])
    #    except:
    #        print 'Problem with:'
    #        print image
    #'''


CLOUD015
20.0001
/Volumes/scratch/lab_menche/Images_CLOMP/Images/1829015__2018-08-14T00_26_31-Measurement1/Images/r07c17f01p01rc1-ch1sk1fk1fl1.tiff
/Volumes/scratch/lab_menche/Images_CLOMP/Images/1829015__2018-08-14T00_26_31-Measurement1/Images/r07c17f02p01rc1-ch1sk1fk1fl1.tiff
/Volumes/scratch/lab_menche/Images_CLOMP/Images/1829015__2018-08-14T00_26_31-Measurement1/Images/r07c17f03p01rc1-ch1sk1fk1fl1.tiff
/Volumes/scratch/lab_menche/Images_CLOMP/Images/1829015__2018-08-14T00_26_31-Measurement1/Images/r07c17f04p01rc1-ch1sk1fk1fl1.tiff
/Volumes/scratch/lab_menche/Images_CLOMP/Images/1829032__2018-08-22T17_53_47-Measurement1/Images/r04c01f01p01rc1-ch1sk1fk1fl1.tiff
/Volumes/scratch/lab_menche/Images_CLOMP/Images/1829032__2018-08-22T17_53_47-Measurement1/Images/r04c01f02p01rc1-ch1sk1fk1fl1.tiff
/Volumes/scratch/lab_menche/Images_CLOMP/Images/1829032__2018-08-22T17_53_47-Measurement1/Images/r04c01f03p01rc1-ch1sk1fk1fl1.tiff
/Volumes/scratch/lab_menche/Images_CLOMP/Images/1829032__2018-08-2

In [ ]:
    print 'Im here

In [ ]:
    print 'Im here

In [ ]:
    print 'Im here

In [ ]:
    print 'Im here

In [ ]:
    print 'Im here

In [ ]:
    print 'Im here

In [ ]:
    print 'Im here